In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tableone import TableOne
import os
import re
import Levenshtein as lev
from fuzzywuzzy import fuzz
import usaddress

# To Show All Columns in DataFrame or numpy arrays
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
np.set_printoptions(threshold=np.inf)

# To Get Rid of Warnings
import warnings
warnings.filterwarnings("ignore")

In [14]:
# PatAddressDim

pataddressdim = pd.read_csv('/home/idies/workspace/Storage/ccai6/NLP in DR/Geocoding/Input/PatAddressDim.csv', encoding='latin-1')
pataddressdim['Address Start Date']=pd.to_datetime(pataddressdim['Address Start Date'], format='%m/%d/%Y')
pataddressdim['EndDate']=pd.to_datetime(pataddressdim['EndDate'], format='%m/%d/%Y')

pataddressdimcol ={'EnterpriseId': "e_mrn",
 'Address Start Date': "address_start",
 'EndDate': "address_end",
 'Address': "address",
 'City': "city",
 'stateorprovinceabbreviation': "state",
 'SourcePostalCode': "zipcode",
 'ZipPlus4_X': "zip4",
 'Longitude': "longitude",
 'Latitude': "latitude",
 'census Tract_X': "census_15",
 'National Deprivation Index': 'adi_nat',
 'State Deprivation Index': "adi_state"}

pataddressdim = pataddressdim.rename(columns=pataddressdimcol)

pataddressdim['status']=pataddressdim['status'].str.strip()

In [15]:
regSub1 = r'\broad\b|\bstreet\b|\bparkway\b|\bdrive\b|\bavenue\b|\bcourt\b|\bcircle\b|\bhighway\b|\bterrace\b'
regSub2 = r'\bnorth\b|\beast\b|\bsouth\b|\bwest\b'
regSkip = r'po.{,5}box|international|homeless|no.{,8}address|unknown|does not have|bad address|\bunk\b'

def abbrev1(matchobj):
    if matchobj.group(0) == 'road': return 'rd'
    elif matchobj.group(0) == 'street': return 'st'
    elif matchobj.group(0) == 'parkway': return 'pkwy'
    elif matchobj.group(0) == 'drive': return 'dr'
    elif matchobj.group(0) == 'avenue': return 'ave'
    elif matchobj.group(0) == 'court': return 'ct'
    elif matchobj.group(0) == 'circle': return 'cir'
    elif matchobj.group(0) == 'place': return 'pl'
    elif matchobj.group(0) == 'highway': return 'hwy'
    elif matchobj.group(0) == 'terrace': return 'ter'
    else: return '-'

def abbrev2(matchobj):
    if matchobj.group(0) == 'north': return 'N'
    elif matchobj.group(0) == 'east': return 'E'
    elif matchobj.group(0) == 'south': return 'S'
    elif matchobj.group(0) == 'west': return 'W'    
    else: return '-'
    
def parse(tokB, add): #parse out the components of the tokenized address     
    global addDf
    num = True
    name = True
    post = True
    hits = 0
    if "AddressNumber" in tokB[0]: 
        AddNumB = tokB[0]["AddressNumber"]
        hits = hits + 1        
    else: #if theres no address number, then separate all numbers preceding letters and try again       
        add = re.sub(r"([0-9])([a-zA-Z])",r"\1 \2", add)
        tokB = usaddress.tag(add)
        if "AddressNumber" in tokB[0]:
            AddNumB = tokB[0]["AddressNumber"]
            hits = hits + 1
        else:
            num = False
            AddNumB = ""
         
    if "StreetNamePreDirectional" in tokB[0]: #e.g. N, S, E, W
        PreDirectB = tokB[0]["StreetNamePreDirectional"] + " "
        hits = hits + 1
    else:
        PreDirectB = ""
    
    if "StreetNamePostType" in tokB[0]:
        PostTpB = tokB[0]["StreetNamePostType"]
        hits = hits + 1
    else: # if post type is not found, then separate post types that are connected to the st name before them. Then remove anything following the post type
        add = re.sub(r"([a-zA-Z])(road\b|street\b|parkway\b|drive\b|lane\b|avenue\b|court\b|circle\b|ln\b|way\b|rd\b|st\b|pkwy\b|dr\b|ave\b|ct\b|cir\b|pl\b)",r"\1 \2", add)        
        add = re.sub(r"(\broad|\bstreet|\bparkway|\bdrive|\bavenue|\bcourt|\bway|\blane).*",r"\1", add)        
        tokB = usaddress.tag(add)        
        if "StreetNamePostType" in tokB[0]:
            PostTpB = tokB[0]["StreetNamePostType"]
            hits = hits + 1
        else:
            post = False
            PostTpB = ""    
    if "StreetName" in tokB[0]: #after all the above modifications, then check for street name
        StNmB = tokB[0]["StreetName"]
        hits = hits + 1
    else:
        StNmB = ""
        name = False
        
    if name:
        finAdd = AddNumB + " " + PreDirectB + StNmB + " "+ PostTpB
    else:
        finAdd = add
    return finAdd, AddNumB, PreDirectB, StNmB, PostTpB, hits
    
def processAdd(add):
    global addDf #for debugging
    finAdd = add
    AddNumB = None
    PreDirectB = None
    StNmB = None 
    PostTpB = None
    hits = 0
    if type(add) == str: #standardize the string first       
        add = add.lower()
        add = re.sub(r"([a-zA-Z])([0-9])",r"\1 \2", add) #separate letters directly preceding numbers
        add = re.sub(r"^([0-9]+)([a-zA-Z])",r"\1 \2", add)#only separates numbers preceding letters at the start
        add = re.sub(r"(.*)(\bapt\b|\bunit\b|#|\broom\b|\btrailer\b).{,5}$",r"\1", add, re.IGNORECASE) #remove apt and related tags at the end of the address
        add = re.sub(r"(.*)(\bapt\b|\bunit\b)\s?(\d+|[A-Za-z] )*(.*)",r"\1 \4", add, re.IGNORECASE) #remove apt and unit from the middle of addresses
        add = re.sub(regSub1, abbrev1, add, re.IGNORECASE) #abbreviate post types
        add = re.sub(regSub2, abbrev2, add, re.IGNORECASE) #abbreviate pre-direction
        try:
            tokB = usaddress.tag(add)                        
            finAdd, AddNumB, PreDirectB, StNmB, PostTpB, hits = parse(tokB, add)                        
        except: #if initial split yields duplicates, then remove everything following full post-type                        
            add = re.sub(r"(\broad\b|\bstreet\b|\bparkway\b|\bdrive\b|\bavenue\b|\bcourt\b|\bway\b|\blane\b).*",r"\1", add)#remove everything after the post                        
            try:
                tokB = usaddress.tag(add)
                finAdd, AddNumB, PreDirectB, StNmB, PostTpB, hits = parse(tokB, add)
            except: #if there are still duplicates, then removed everything following abbreviated post-type           
                add = re.sub(r"(rd\b|st\b|pkwy\b|dr\b|ave\b|ct\b|cir\b|ln\b).*$",r"\1", add)#remove everything after the post
                addDf = True
                try:
                    tokB = usaddress.tag(add)
                    finAdd, AddNumB, PreDirectB, StNmB, PostTpB, hits = parse(tokB, add)                
                except:                       
                    finAdd = add
                    hits = 0
    return finAdd, AddNumB, PreDirectB, StNmB, PostTpB, hits


In [32]:
df = pataddressdim.head(30000)
#-----For testing specific addresses--------
#df.at[0,'address'] = "2010 benlow st"
#df.at[1,'address'] = "2010 bentalou st"
#df.at[2,'address'] = "2010 benlow st"

df2 = df
currMrn = df.iloc[0]['e_mrn']
origAdd = df.iloc[0]['address']
endAdd, endAddNum, endPreDirect, endStNm, endPostTp, currHits = processAdd(origAdd)
endStat = df.iloc[0]['status']
endOrigAdd = df.iloc[0]['address']
endRow =  df.iloc[0]['address_end']
j = 0

addDf = False #for debugging

for i in range(df.shape[0]):
    addDf = False
    if i % 10000 == 0: #to show progress
        print(i)
    
    #obtain the relavent data for each row
    origAdd = df.iloc[i]['address']
    fip = df.iloc[i]['address']
    rowAdd, AddNum, PreDirect, StNm, PostTp, hits = processAdd(origAdd)
    rowStat = df.iloc[i]['status']
    
    #reset the variables for address matching
    splitMatch = "Z"
    Match = False
    Ratio = 0
    Ratio2 = 0
    Fuzz = 0
    nRat = 0
    
    if StNm and endStNm and AddNum and endAddNum: #start by matching the individual address parts
        Ratio = lev.ratio(StNm, endStNm)
        Fuzz = fuzz.partial_ratio(StNm, endStNm)
        nRat = lev.ratio(AddNum, endAddNum)
        if  (Ratio >= 0.75 and nRat >= 0.7): #different combinations of street name and address number matches
            splitMatch = "A"            
        elif (Ratio >= 0.7 and nRat >= 0.75):
            splitMatch = "B"
        elif (Ratio >= 0.65 and nRat >= 0.9):
            splitMatch = "C"
        elif (Fuzz >= 75 and nRat >= 0.9):
            splitMatch = "D"        
    elif type(rowAdd) == str and type(endAdd) == str: #if the address can't be split into parts, see if it is still an overall match
        Ratio2 = lev.ratio(rowAdd, endAdd)
        if  Ratio2 > 0.8:
            Match = True
            
    if i == 0:        
        continue
    elif df.iloc[i]['e_mrn'] == currMrn and (Match or splitMatch == "A" or splitMatch == "B" or splitMatch == "C" or splitMatch == "D"): #if same address                       
        endRow = df.iloc[i]['address_end']            
        if (endStat != "Street" and rowStat == "Street") or hits > currHits: #update address if it is more complete/street matched 
            endAdd = rowAdd           
            endAddNum = AddNum
            endPreDirect = PreDirect
            endStNm = StNm
            endPostTp = PostTp
            currHits = hits
            endOrigAdd = origAdd
            endStat = rowStat
        df2 = df2.drop(i)                        
    else:        #for different addresses, save the final address information
        df2.at[j,'address_end'] = endRow        
        df2.at[j,'address'] = endAdd
        df2.at[j,'adi_state'] = endOrigAdd
        num = str(df2.at[j,'zipcode'])
        if "-" in num: #remove hyphens so that SAS can run properly
            df2.at[j,'zipcode'] = num.split("-", 1)[0]                                        
        j = i
            
        currMrn = df.iloc[i]['e_mrn']#reset all the variables
        currHits = hits
        endStat = rowStat
        endAdd = rowAdd
        endAddNum = AddNum
        endPreDirect = PreDirect
        endStNm = StNm
        endPostTp = PostTp
        endRow = df.iloc[i]['address_end']
        endOrigAdd = origAdd        

#set final address details
df2.at[j,'address_end'] = endRow
df2.at[j,'address'] = endAdd
df2.at[j,'adi_state'] = endOrigAdd

#remove unecesarry columns
df3 = df2.loc[:, ~df2.columns.isin(['address_start', 'address_end', 'zip4', 'longitude', 'latitude', 'adi_nat'])]

0
10000
20000


In [33]:
df3.to_csv('/home/idies/workspace/Storage/ccai6/NLP in DR/Geocoding/Output/SAS_input_3_18_middleRemoved_Full.csv')#includes indexes